In [ ]:
%pylab inline
rcParams['figure.figsize'] = (16, 5)

In [ ]:
from scipy.io import wavfile
from IPython.display import Audio

In [ ]:
sr, sig = wavfile.read('media/superstition.wav')
left = sig[67000:110000,0]
plot(left)
Audio(data=left, rate=sr)

## [Bessel Function](https://en.wikipedia.org/wiki/Bessel_function)

![](https://upload.wikimedia.org/wikipedia/commons/5/5d/Bessel_Functions_%281st_Kind%2C_n%3D0%2C1%2C2%29.svg)

Let's convolve our signal with an arbitrary function. We'll use a Bessel....

In [ ]:
from scipy.special import jn # jn is a bessel function...

In [ ]:
x = linspace(0, 20, 1000)
j1 = jn(1, x)
plot(x, j1)
pass

In [ ]:
c = convolve(left, j1[::-1])
plot(c)
Audio(data=c, rate=sr)

In [ ]:
# ::-1 slice notation means "make a reversed copy"
plot(abs(fft.rfft(j1[::-1], n=len(j1) * 4)))
xlim((0, 300))
pass

In [ ]:
plot(abs(fft.rfft(left)))
xlim((0, 3000))
pass

In [ ]:
plot(abs(fft.rfft(c)))
xlim((0, 3000))
pass

In [ ]:
Audio(data=c, rate=sr)

Seems to be a lowpass filter. Let's convolve with some high-passed noise...

In [ ]:
noise = (random.random(1024) * 2) -1
hp_noise = noise[1:] - noise[:-1]
plot(abs(fft.rfft(hp_noise)))
pass

In [ ]:
c = convolve(left, hp_noise[::-1])
plot(c)
Audio(data=c, rate=sr)

This seems to be a highpass filter.

## [Ricker Wavelet](https://en.wikipedia.org/wiki/Mexican_hat_wavelet)

![](https://upload.wikimedia.org/wikipedia/commons/0/08/MexicanHatMathematica.svg)

$$\psi(t) = {2 \over {\sqrt {3\sigma}\pi^{1 \over 4}}} \left( 1 - {({t \over \sigma})^2} \right) e^{{-1 \over 2}({t \over \sigma})^2}$$

([blob detection](https://en.wikipedia.org/wiki/Blob_detection#The_Laplacian_of_Gaussian), [David Marr][])

[David Marr]: https://en.wikipedia.org/wiki/David_Marr_(neuroscientist)

Now let's convolve our signal with something new.

In [ ]:
from scipy.signal import ricker, freqz

In [ ]:
points = 100
a = 1
vec2 = ricker(points, a)
stem(vec2)
pass

In [ ]:
wlet = ricker(8000, 10)
cwt = convolve(left, wlet, mode='same') # depends on mode
plot(cwt)
Audio(data=cwt, rate=sr)

In [ ]:
semilogx(abs(rfft(cwt)))
pass

In [ ]:
freqz(ricker(8000, 32), plot=lambda w, h: plot(w, abs(h)))
freqz(ricker(8000, 16), plot=lambda w, h: plot(w, abs(h)))
freqz(ricker(8000,  8), plot=lambda w, h: plot(w, abs(h)))
freqz(ricker(8000,  4), plot=lambda w, h: plot(w, abs(h)))
freqz(ricker(8000,  2), plot=lambda w, h: plot(w, abs(h)))
freqz(ricker(8000,  1), plot=lambda w, h: plot(w, abs(h)))
title('Frequency Response of various Ricker wavelets')
pass

In [ ]:
freqz(ricker(8000, 6), plot=lambda w, h: plot(w, abs(h)))
freqz(ricker(8000, 5), plot=lambda w, h: plot(w, abs(h)))
freqz(ricker(8000, 4), plot=lambda w, h: plot(w, abs(h)))
freqz(ricker(8000, 3), plot=lambda w, h: plot(w, abs(h)))
freqz(ricker(8000, 2), plot=lambda w, h: plot(w, abs(h)))
freqz(ricker(8000, 1), plot=lambda w, h: plot(w, abs(h)))
title('Frequency Response of various Ricker wavelets')
pass

Each is a band pass filter!

In [ ]:
phs = linspace(0, 10 * 2 * pi, 256, endpoint=False)

complex_phasor = 1j*sin(phs) + cos(phs)
freqz(complex_phasor, plot=lambda w, h: plot(w, abs(h)))

complex_phasor2 = 1j*sin(2 * phs) + cos(2 * phs)
freqz(complex_phasor2, plot=lambda w, h: plot(w, abs(h)))

complex_phasor3 = 1j*sin(4 * phs) + cos(4 * phs)
freqz(complex_phasor3, plot=lambda w, h: plot(w, abs(h)))

complex_phasor4 = 1j*sin(8 * phs) + cos(8 * phs)
freqz(complex_phasor4, plot=lambda w, h: plot(w, abs(h)))

title('Frequency Response of Complex Phasors (FFT)')

pass

In [ ]:
phs = linspace(0, 10 * 2 * pi, 256, endpoint=False)

complex_phasor = 1j*sin(phs) + cos(phs)
freqz(complex_phasor, plot=lambda w, h: plot(w, abs(h)))

complex_phasor2 = 1j*sin(2 * phs) + cos(2 * phs)
freqz(complex_phasor2, plot=lambda w, h: plot(w, abs(h)))

complex_phasor3 = 1j*sin(3 * phs) + cos(3 * phs)
freqz(complex_phasor3, plot=lambda w, h: plot(w, abs(h)))

complex_phasor4 = 1j*sin(4 * phs) + cos(4 * phs)
freqz(complex_phasor4, plot=lambda w, h: plot(w, abs(h)))

title('Frequency Response of Complex Phasors (FFT)')

pass

So complex phasors are also band pass filters, but of a different character; They have **constant bandwidth** whereas Ricker wavelets each have a different bandwidth that seems to be based on their *width* parameter. 

A quick aside on calculating the frequency of an FFT bin...

In [ ]:
binNumber = 1
fftSize = 256
normalizedFrequency = binNumber / fftSize
normalizedFrequency

In [ ]:
sampleRate = 44100
frequencyHertz = sampleRate * normalizedFrequency
frequencyHertz

In [ ]:
def frequency_of_bin(binNumber, fftSize, sampleRate):
    return sampleRate * binNumber / fftSize

f = frequency_of_bin(1, 256, 44100)
F = fftfreq(256, 1/44100)[1]

f, F

In [ ]:
wlet = ricker(8000, 100)
cwt = convolve(left, wlet, mode='same')
subplot(121)
plot(cwt)
subplot(122)
plot(wlet)
Audio(data=cwt, rate=sr)

In [ ]:
wlet = ricker(8000, 800)
cwt = convolve(left, wlet, mode='same')
subplot(121)
plot(cwt)
subplot(122)
plot(wlet)
Audio(data=cwt, rate=sr)

Each Ricker represents a band-pass filter. Using many of these, we can desconstruct a signal into bands and perfectly reconstruct the signal without loss of information. Wavelets are an alternative to the Fourier Transform.

# The Wavelet Transform

$$X(a,b) = \frac{1}{\sqrt{a}}\int_{-\infty}^{\infty}\overline{\Psi\left(\frac{t - b}{a}\right)} x(t)\, dt$$

The cross-correlation of the wavelet with the signal. The wavelet transform is a function of the wavelet's shift ($b$) and scaling ($a$)

The wavelet function is chosen so that it forms a complete orthonormal system. This means that, like the Fourier Transform, we may reconstruct the original signal perfectly without loss of information.

## [Scaleogram](https://en.wikipedia.org/wiki/Scaleogram)s and the [Continuous Wavelet Transform](https://en.wikipedia.org/wiki/Continuous_wavelet_transform)

 Also see [Scale-Based Analysis](https://www.mathworks.com/help/wavelet/gs/continuous-wavelet-transform-and-scale-based-analysis.html).
 
![](https://upload.wikimedia.org/wikipedia/commons/5/58/Scaleogram.png)

In [ ]:
from scipy.signal import cwt # Continuous Wavelet Transform 

In [ ]:
width = arange(1, 30) * 10 # leave some gaps for efficiency (we are only visualizing)
cwtmatr = cwt(left.astype('float64'), ricker, width)
width

In [ ]:
# the verticle is the wavelet "width" (or scale) parameter which we can think of as relating to frequency
# the horizontal is "shift" which we can think of as time
# the color indicates the energy
imshow(cwtmatr, extent=[-1, 1, 1, 300], cmap='PRGn', aspect='auto',
       vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max(), interpolation='nearest')
colorbar()
shape(cwtmatr)

In [ ]:
plot(left) # compare to...
Audio(data=left, rate=sr)

In [ ]:
width_100 = cwtmatr[9, :]
plot(width_100)
Audio(data=width_100/1000, rate=sr)

In [ ]:
wlet = ricker(len(left), 100)
cwt_ = convolve(left, wlet, mode='same')
plot(cwt_)
Audio(data=width_100/1000, rate=sr)

Note that the two methods above are equivalent!

In [ ]:
# zoom in on the front part
print(shape(cwtmatr))
imshow(cwtmatr[:, :10000], extent=[-1, 1, 1, 300], cmap='PRGn', aspect='auto',
       vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max(), interpolation='nearest')
pass

In [ ]:
# zoom in on the back part
imshow(cwtmatr[:, -11000:], extent=[-1, 1, 1, 3000], cmap='PRGn', aspect='auto',
       vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max(), interpolation='nearest')
pass

In [ ]:
ha = plt.subplot(111)
specgram(left, Fs=sr)
ha.set_yscale('symlog')
ylim(10, 10000)
pass

In [ ]:
width = arange(1, 10) / 2.0 # small widths == high frequencies
cwtmatr = cwt(left.astype(float64), ricker, width)
width

In [ ]:
imshow(cwtmatr, extent=[-1, 1, 1, 10], cmap='PRGn', aspect='auto',
       vmax=abs(cwtmatr).max(), vmin=-abs(cwtmatr).max(), interpolation='nearest')
shape(cwtmatr)

In [ ]:
plot(cwtmatr[2,:])
Audio(data=cwtmatr[2,:], rate=sr)

Zoom in to the highhat

In [ ]:
cwt_slice = cwtmatr[:,15000:30000]
imshow(cwt_slice, extent=[-1, 1, 1, 300], cmap='PRGn', aspect='auto',
       vmax=abs(cwt_slice).max(), vmin=-abs(cwt_slice).max(), interpolation='nearest')
pass

> Wavelets are defined by the wavelet function ψ(t) (i.e. the mother wavelet) and scaling function φ(t) (also called father wavelet) in the time domain.

> The wavelet function is in effect a band-pass filter and scaling it for each level halves its bandwidth. This creates the problem that in order to cover the entire spectrum, an infinite number of levels would be required. The scaling function filters the lowest level of the transform and ensures all the spectrum is covered.

> The Haar transform is the simplest of the wavelet transforms. This transform cross-multiplies a function against the Haar wavelet with various shifts and stretches, like the Fourier transform cross-multiplies a function against a sine wave with two phases and many stretches.


(Wikipedia on [Haar wavelet](https://en.wikipedia.org/wiki/Haar_wavelet) and [Wavelet](https://en.wikipedia.org/wiki/Wavelet))

## The [Discrete Wavelet Transform](https://en.wikipedia.org/wiki/Discrete_wavelet_transform)

We've been using the `cwt` function in `scipy.signal` to do the "[Continuous Wavelet Transform (CWT)](https://en.wikipedia.org/wiki/Continuous_wavelet_transform)" but of course it is discrete, so what's up? The `cwt` is a discretization of the CWT _per sample_. It calculates the result of each wavelet for each sample of the signal. This is costly.

The Discrete Wavelet Transform (DWT) refers to broader discretization in time and scale, i.e. bigger chunks. Not all scales and shifts need to be computed in order to achieve a reversible transform. The DWT minimizes the computational cost while still offering a reversable transform.

> the basis function $\psi$ can be regarded as an impulse response of a system with which the function $x(t)$ has been filtered. The transformed signal provides information about the time and the frequency. Therefore, wavelet-transformation contains information similar to the short-time-Fourier-transformation, but with additional special properties of the wavelets (https://en.wikipedia.org/wiki/Wavelet_transform#Principle)

$$\Delta t \Delta \omega \geqq \frac{1}{2}$$

> where $t$ represents time and $\omega$ angular frequency ($\omega = 2 \pi f$, where $f$ is temporal frequency). The higher the required resolution in time, the lower the resolution in frequency has to be. The larger the extension of the analysis windows is chosen, the larger is the value of $\Delta t$. (https://en.wikipedia.org/wiki/Uncertainty_principle#Signal_processing)


`scipy` does not have a DWT library yet, so we use [pywt (aka PyWavelets)](https://pywavelets.readthedocs.io/en/latest). Follow the [installation instructions](https://pywavelets.readthedocs.io/en/latest/#install) for your system.

(See also [FWT](https://en.wikipedia.org/wiki/Fast_wavelet_transform), [Ingrid Daubechies](https://en.wikipedia.org/wiki/Ingrid_Daubechies), [Simple Haar Wavelet Transform Implementation](https://en.wikipedia.org/wiki/Discrete_wavelet_transform#Code_example))

In [ ]:
from pywt import wavedec, waverec, dwt

- http://wavelets.pybytes.com/wavelet/db1
- http://wavelets.pybytes.com/wavelet/haar
<img src="https://upload.wikimedia.org/wikipedia/commons/a/a0/Haar_wavelet.svg" alt="" width=600>
![](https://upload.wikimedia.org/wikipedia/commons/c/c4/STFT_and_WT.jpg)

In [ ]:
tree = wavedec(left, 'db1') # Daubechies 1 == Haar

In [ ]:
len(tree)

In [ ]:
tree[0]

In [ ]:
for level in range(1, len(tree)):
    print(len(tree) - level, len(tree[level]))

In [ ]:
tree[1]

In [ ]:
tree[2]

In [ ]:
tree[3]

In [ ]:
tree[4]

In [ ]:
# From: http://stackoverflow.com/questions/16482166/basic-plotting-of-wavelet-analysis-output-in-matplotlib
data = tree
bottom = 0

vmin = min(map(lambda x: -max(abs(x)), data))
vmax = max(map(lambda x: max(abs(x)), data))

for row in range(0, len(data)):
    scale = 2.0 ** (row - len(data))

    imshow(
        array([data[row]]),
        interpolation = 'nearest',
        vmin = vmin,
        vmax = vmax,
        extent = [0, 1, bottom, bottom + scale],
        cmap='PRGn', aspect='auto')

    bottom += scale

ylim((0, 0.2))
colorbar()
pass

In [ ]:
data = tree
bottom = 0

vmin = min(map(lambda x: -max(abs(x)), data))
vmax = max(map(lambda x: max(abs(x)), data))

for row in range(0, len(data)):
    scale = 2.0 ** (row - len(data))

    imshow(
        array([data[row]]),
        interpolation = 'nearest',
        vmin = vmin,
        vmax = vmax,
        extent = [0, 1, bottom, bottom + scale],
        cmap='PRGn', aspect='auto')

    bottom += scale

ylim((0, 0.1))
xlim((0, 0.1)) # zoom in
colorbar()
pass

In the DWT the "smaller" wavelets are performed more often in time and occupy a wider "detail" region. This is why the DWT can resolve detail accurately both in "detail" and "time".

# 2D DWT

In [ ]:
from pywt import dwt2, idwt2

In [ ]:
im = imread('media/fruits.png').sum(axis=2)

In [ ]:
im.dtype

In [ ]:
imshow(im, cmap=cm.gray)
pass

In [ ]:
tree = dwt2(im, 'db1')

In [ ]:
len(tree)

In [ ]:
shape(tree)

In [ ]:
array(tree[0]).shape

In [ ]:
shape(im)

In [ ]:
len(tree[1])

In [ ]:
array(tree[1][0]).shape

In [ ]:
# coefficients
cA = tree[0] # approximation
cH = array(tree[1])[0] # horizontal detail
cV = array(tree[1])[1] # vertical detail
cD = array(tree[1])[2] # diagonal detail
shape(cD)

In [ ]:
subplot(121)
imshow(im, cmap=cm.gray, interpolation='nearest')
colorbar()
subplot(122)
imshow(cA, cmap=cm.gray, interpolation='nearest')
colorbar()
pass

In [ ]:
imshow(cH, cmap=cm.RdGy, interpolation='nearest')
colorbar()
pass

In [ ]:
imshow(cV, cmap=cm.RdGy, interpolation='nearest')
colorbar()
pass

In [ ]:
imshow(cD, cmap=cm.RdGy, interpolation='nearest')
colorbar()
pass

In [ ]:
imshow(idwt2((cA, (cH, cH, cD)), 'db1'), cmap=cm.gray)
pass

## Multi-level transform

In [ ]:
from pywt import wavedec2, waverec2

In [ ]:
tree = wavedec2(im, 'db1')

In [ ]:
len(tree) # 10 levels

In [ ]:
array(tree[0]).shape

In [ ]:
array(tree[1]).shape

In [ ]:
array(tree[2]).shape

In [ ]:
array(tree[3]).shape

In [ ]:
# approximation coefficients
print('10', shape(tree[0]))

# detail coefficients
for level in range(1, len(tree)):
    print(len(tree) - level, shape(tree[level]))
# detail coefficients
# 0:cHn (horizontal)
# 1:cVn (verticle)
# 2:cDn (diagonal)

In [ ]:
imshow(array(tree[0]), cmap=cm.RdGy, interpolation='nearest')
colorbar()
array(tree[0]).shape

That's a first approximation of the image!!! 

In [ ]:
imshow(array(tree[2][0]), cmap=cm.RdGy, interpolation='nearest')
colorbar()
array(tree[2][0]).shape

In [ ]:
imshow(array(tree[3][0]), cmap=cm.RdGy, interpolation='nearest')
colorbar()
array(tree[3][0]).shape

In [ ]:
imshow(array(tree[4][0]), cmap=cm.RdGy, interpolation='nearest')
colorbar()
array(tree[4][0]).shape

In [ ]:
imshow(array(tree[5][0]), cmap=cm.RdGy, interpolation='nearest')
colorbar()
array(tree[5][0]).shape

In [ ]:
imshow(array(tree[6][0]), cmap=cm.RdGy, interpolation='nearest')
colorbar()
array(tree[6][0]).shape

In [ ]:
imshow(array(tree[9][0]), cmap=cm.RdGy, interpolation='nearest')
colorbar()
array(tree[9][0]).shape

In [ ]:
imshow(array(tree[9][0]), cmap=cm.RdGy, interpolation='nearest')
colorbar()
ylim((250, 150))
xlim((200, 300))
array(tree[9][0]).shape

In [ ]:
imshow(array(tree[9][1]), cmap=cm.RdGy, interpolation='nearest')
colorbar()
ylim((250, 150))
xlim((200, 300))
array(tree[9][1]).shape

In [ ]:
imshow(array(tree[9][2]), cmap=cm.RdGy, interpolation='nearest')
colorbar()
ylim((250, 150))
xlim((200, 300))
array(tree[9][2]).shape

In [ ]:
from scipy.misc import imresize

In [ ]:
scaling1 = array(tree[1][0])
scaling2 = array(tree[2][0])
summed = scaling2 + imresize(scaling1, scaling2.shape, interp='nearest' )
imshow(summed, interpolation='nearest', cmap=cm.gray)
pass

In [ ]:
size = array(tree[9][0]).shape

num = 3
summed = imresize(array(tree[0]), size, interp='nearest').astype(float)

for i in range(num - 1):
    scaled = imresize(array(tree[i + 1][0]), size, interp='nearest' )
    summed += scaled
    
imshow(summed, interpolation='nearest', cmap=cm.gray)
pass

In [ ]:
size = array(tree[9][0]).shape

num = 5
summed = imresize(array(tree[0]), size, interp='nearest').astype(float)

for i in range(num - 1):
    scaled = imresize(array(tree[i + 1][0]), size, interp='nearest' )
    summed += scaled
    
imshow(summed, interpolation='nearest', cmap=cm.gray)
pass

In [ ]:
size = array(tree[9][0]).shape

num = 10
summed = imresize(array(tree[0]), size, interp='nearest').astype(float)

for i in range(num - 1):
    scaled = imresize(array(tree[i + 1][0]), size, interp='nearest' ).astype(float)
    summed += scaled
    
imshow(summed, interpolation='nearest', cmap=cm.gray)
pass

In [ ]:
size = array(tree[9][0]).shape

num = 10
summed = imresize(array(tree[0]), size, interp='nearest').astype(float)

for i in range(num - 1):
    scaled = imresize(array(tree[i + 1][1]), size, interp='nearest' )
    summed += scaled
    
imshow(summed, interpolation='nearest', cmap=cm.gray)
pass

In [ ]:
size = array(tree[9][0]).shape

num = 10
summed = imresize(array(tree[0]), size, interp='nearest').astype(float)

for i in range(num - 1):
    scaled = imresize(array(tree[i + 1][2]), size, interp='nearest' )
    summed += scaled
    
imshow(summed, interpolation='nearest', cmap=cm.gray)
pass

In [ ]:
size = array(tree[9][0]).shape

num = 10
summed = imresize(array(tree[0]), size, interp='nearest').astype(float)

for i in range(num - 1):
    scaled = imresize(array(tree[i + 1][0]), size, interp='nearest' )
    scaled *= imresize(array(tree[i + 1][1]), size, interp='nearest' )
    scaled *= imresize(array(tree[i + 1][2]), size, interp='nearest' )
    summed += scaled
    
imshow(summed, interpolation='nearest', cmap=cm.gray)
pass

we wish we could manually do the reconstruction above

In [ ]:
# this is the inverse of wavedec2
recon = waverec2(tree, 'db1')
imshow(recon, cmap=cm.gray)
pass

Some uses of wavelets:

* Signal processing: Processing in the wavelet domain (smoothing and denoising) 
* Data compression: Both lossless and lossy
* Edge detection
* Recognition and analysis
* Numerical Analysis: used in the solution of partial differential equations and integral equations

http://cva.stanford.edu/classes/ee482a/docs/lect01_sample.pdf

By: Andrés Cabrera mantaraya36@gmail.com
For MAT course MAT 201A at UCSB

Adapted by Karl Yerkes

This ipython notebook is licensed under the CC-BY-NC-SA license: http://creativecommons.org/licenses/by-nc-sa/4.0/

![http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png](http://i.creativecommons.org/l/by-nc-sa/3.0/88x31.png)